# original functions

In [4]:
# https://stackoverflow.com/questions/53004311/how-to-add-conda-environment-to-jupyter-lab 

import os
import json

import requests

from datetime import date
from datetime import datetime

from dotenv import load_dotenv
import sys

pf_api_url = "https://graphql.probablefutures.org"
pf_token_audience = "https://graphql.probablefutures.com"
pf_token_url = "https://probablefutures.us.auth0.com/oauth/token"

load_dotenv()

def convert_to_iso8601(date_str):
    try:
        # Parse the date string to a datetime object
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        # Format the datetime object to ISO 8601 format with timezone offset
        iso8601_date = date_obj.strftime("%Y-%m-%dT%H:%M:%S+00:00")
        return iso8601_date
    except ValueError:
        # Return the original string if it's not in the expected date format
        return date_str

def get_current_datetime():
    return str(date.today())

def get_pf_token():
    client_id = os.getenv("CLIENT_ID")
    client_secret = os.getenv("CLIENT_SECRET")
    response = requests.post(
        pf_token_url,
        json={
            "client_id": client_id,
            "client_secret": client_secret,
            "audience": pf_token_audience,
            "grant_type": "client_credentials",
        },
    )
    access_token = response.json()["access_token"]
    return access_token

def get_pf_data(address, country, warming_scenario="1.5"):
    variables = {}

    location = f"""
        country: "{country}"
        address: "{address}"
    """

    query = (
        """
        mutation {
            getDatasetStatistics(input: { """
        + location
        + """ \
                    warmingScenario: \"""" + warming_scenario + """\" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    """
    )
    print(query)

    access_token = get_pf_token()
    url = pf_api_url + "/graphql"
    headers = {"Authorization": "Bearer " + access_token}
    response = requests.post(
        url, json={"query": query, "variables": variables}, headers=headers
    )
    return str(response.json())

resp = get_pf_data(address="Mombassa", country="Kenya", warming_scenario="1.5")
resp = resp.replace("'", '"')
print(json.dumps(json.loads(resp), indent=2))


        mutation {
            getDatasetStatistics(input: { 
        country: "Kenya"
        address: "Mombassa"
                         warmingScenario: "1.5" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


KeyError: 'access_token'

In [3]:
import pandas as pd
dwata = json.loads(resp)
pd.DataFrame(dwata['data']['getDatasetStatistics']['datasetStatisticsResponses'])

NameError: name 'resp' is not defined

In [1]:
data = pd.DataFrame(dwata['data']['getDatasetStatistics']['datasetStatisticsResponses'])

NameError: name 'pd' is not defined

# function to process JSON output dataframe

In [4]:
import json

# Convert the JSON data to a DataFrame
def json_to_dataframe(json_data, city, country):
    # Extract the relevant part of the JSON data
    json_data = json.loads(json_data)
    data = json_data['data']['getDatasetStatistics']['datasetStatisticsResponses']
    # Convert to a DataFrame
    df = pd.DataFrame(data)
    # Normalize the 'info' column if needed
    if not df['info'].apply(lambda x: x == {}).all():
        info_df = pd.json_normalize(df['info'])
        df = df.drop(columns=['info']).join(info_df)
    df['city'] = city
    df['country'] = country
    return df


In [6]:
# Parse the JSON string
#parsed_json = json.loads(resp)#json_data)

# Call the function and display the dataframe
df = json_to_dataframe(resp, "Mombassa", "Kenya")
df.head()


,datasetId,midValue,name,unit,warmingScenario,latitude,longitude,climateZoneName,city,country
0,40601,30.0,Change in total annual precipitation,mm,1.5,-4,39.6,NaN,Mombassa,Kenya
1,40616,70.0,Change in wettest 90 days,mm,1.5,-4,39.6,NaN,Mombassa,Kenya
2,40607,21.0,Change in dry hot days,days,1.5,-4,39.6,NaN,Mombassa,Kenya
3,40614,0.0,Change in snowy days,days,1.5,-4,39.6,NaN,Mombassa,Kenya
4,40612,1.0,Change in frequency of “1-in-100-year” storm,x as frequent,1.5,-4,39.6,NaN,Mombassa,Kenya


In [7]:
df.columns

Index(['datasetId', 'midValue', 'name', 'unit', 'warmingScenario', 'latitude',
       'longitude', 'climateZoneName', 'city', 'country'],
      dtype='object')

In [8]:
columns = ['datasetId', 'midValue', 'name', 'unit', 'warmingScenario', 'latitude',
       'longitude', 'climateZoneName', 'city', 'country']

In [9]:
df.shape

(27, 10)

## testing different locations iteratively

In [81]:
locations = [['denver','usa'],['kyoto', 'japan'],['miami','usa'],['london','england'],['hamburg','germany']]

In [87]:
output = pd.DataFrame(columns=columns)
#output

for location in locations:
    resp = get_pf_data(address=location[0], country=location[1], warming_scenario="2")
    resp = resp.replace("'", '"')
    location_data = json_to_dataframe(resp, city=location[0], country=location[1])
    output = pd.concat([output, location_data], axis=0)


        mutation {
            getDatasetStatistics(input: { 
        country: "usa"
        address: "denver"
                         warmingScenario: "2" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


/var/folders/9f/7gjvlvl139147dwq2vt94d8c0000gp/T/ipykernel_1651/1723902196.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = pd.concat([output, location_data], axis=0)



        mutation {
            getDatasetStatistics(input: { 
        country: "japan"
        address: "kyoto"
                         warmingScenario: "2" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    

        mutation {
            getDatasetStatistics(input: { 
        country: "usa"
        address: "miami"
                         warmingScenario: "2" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


In [90]:
output = output[['city', 'country', 'name', 'midValue', 'unit' ]]
output = output[output['name'].str.contains('Change')]
output.reset_index(drop=True, inplace=True)

In [92]:
output.head(50)

,city,country,name,midValue,unit
0,denver,usa,Change in total annual precipitation,-7.0,mm
1,denver,usa,Change in wettest 90 days,-2.0,mm
2,denver,usa,Change in dry hot days,14.0,days
3,denver,usa,Change in snowy days,-2.0,days
4,denver,usa,Change in frequency of “1-in-100-year” storm,2.0,x as frequent
5,denver,usa,Change in precipitation “1-in-100-year” storm,19.0,mm
6,denver,usa,Change in wildfire danger days,15.0,days
7,denver,usa,Change in water balance,-0.5,z-score
8,kyoto,japan,Change in total annual precipitation,65.0,mm
9,kyoto,japan,Change in wettest 90 days,52.0,mm


# new pf_data_function which processes output

In [3]:
# Convert the JSON data to a DataFrame
def json_to_dataframe(json_data, address, country):
    # Extract the relevant part of the JSON data
    json_data = json.loads(json_data)
    data = json_data['data']['getDatasetStatistics']['datasetStatisticsResponses']
    # Convert to a DataFrame
    df = pd.DataFrame(data)
    # Normalize the 'info' column if needed
    if not df['info'].apply(lambda x: x == {}).all():
        info_df = pd.json_normalize(df['info'])
        df = df.drop(columns=['info']).join(info_df)
    df['address'] = address
    df['country'] = country
    df = df[['address', 'country', 'name', 'midValue', 'unit' ]]
    df = df[df['name'].str.contains('Change')]
    df.reset_index(drop=True, inplace=True)
    return df

In [4]:
def get_pf_data_new(address, country, warming_scenario="2.0"):
    variables = {}

    location = f"""
        country: "{country}"
        address: "{address}"
    """

    query = (
        """
        mutation {
            getDatasetStatistics(input: { """
        + location
        + """ \
                    warmingScenario: \"""" + warming_scenario + """\" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    """
    )
    print(query)

    access_token = get_pf_token()
    url = pf_api_url + "/graphql"
    headers = {"Authorization": "Bearer " + access_token}
    response = requests.post(
        url, json={"query": query, "variables": variables}, headers=headers
    )
    
    response = str(response.json()).replace("'", '"')

    parsed_output = json_to_dataframe(response, address=address, country=country)
    
    return parsed_output #str(response.json())


In [5]:
location_output = get_pf_data_new(address="Mombassa", country="Kenya")
#resp = resp.replace("'", '"')
#print(json.dumps(json.loads(resp), indent=2))
location_output.head(20)


        mutation {
            getDatasetStatistics(input: { 
        country: "Kenya"
        address: "Mombassa"
                         warmingScenario: "2.0" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


,address,country,name,midValue,unit
0,Mombassa,Kenya,Change in total annual precipitation,15.0,mm
1,Mombassa,Kenya,Change in wettest 90 days,30.0,mm
2,Mombassa,Kenya,Change in dry hot days,29.0,days
3,Mombassa,Kenya,Change in snowy days,0.0,days
4,Mombassa,Kenya,Change in frequency of “1-in-100-year” storm,1.0,x as frequent
5,Mombassa,Kenya,Change in precipitation “1-in-100-year” storm,40.0,mm
6,Mombassa,Kenya,Change in wildfire danger days,3.0,days
7,Mombassa,Kenya,Change in water balance,-0.2,z-score


## testing new iterations of prompt eng

### dropping any rows with no change

In [13]:
def json_to_dataframe_new(json_data, address, country):
    # Extract the relevant part of the JSON data
    json_data = json.loads(json_data)
    data = json_data['data']['getDatasetStatistics']['datasetStatisticsResponses']
    # Convert to a DataFrame
    df = pd.DataFrame(data)
    # Normalize the 'info' column if needed
    if not df['info'].apply(lambda x: x == {}).all():
        info_df = pd.json_normalize(df['info'])
        df = df.drop(columns=['info']).join(info_df)
    df['address'] = address
    df['country'] = country
    df = df[['address', 'country', 'name', 'midValue', 'unit' ]]
    df = df[df['name'].str.contains('Change')]
    df = df[~(df['midValue'] == '0.0')]
    df.reset_index(drop=True, inplace=True)
    return df

In [16]:
def get_pf_data_new(address, country, warming_scenario="2.0"):
    variables = {}

    location = f"""
        country: "{country}"
        address: "{address}"
    """

    query = (
        """
        mutation {
            getDatasetStatistics(input: { """
        + location
        + """ \
                    warmingScenario: \"""" + warming_scenario + """\" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    """
    )
    print(query)

    access_token = get_pf_token()
    url = pf_api_url + "/graphql"
    headers = {"Authorization": "Bearer " + access_token}
    response = requests.post(
        url, json={"query": query, "variables": variables}, headers=headers
    )
    
    response = str(response.json()).replace("'", '"')

    parsed_output = json_to_dataframe_new(response, address=address, country=country)
    
    return parsed_output #str(response.json())


In [34]:
location_output = get_pf_data_new(address="Denver", country="Colorado")
#resp = resp.replace("'", '"')
#print(json.dumps(json.loads(resp), indent=2))
location_output.head(20)


        mutation {
            getDatasetStatistics(input: { 
        country: "Colorado"
        address: "Denver"
                         warmingScenario: "2.0" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


,address,country,name,midValue,unit
0,Denver,Colorado,Change in total annual precipitation,-7.0,mm
1,Denver,Colorado,Change in wettest 90 days,-2.0,mm
2,Denver,Colorado,Change in dry hot days,14.0,days
3,Denver,Colorado,Change in snowy days,-2.0,days
4,Denver,Colorado,Change in frequency of “1-in-100-year” storm,2.0,x as frequent
5,Denver,Colorado,Change in precipitation “1-in-100-year” storm,19.0,mm
6,Denver,Colorado,Change in wildfire danger days,15.0,days
7,Denver,Colorado,Change in water balance,-0.5,z-score


### splitting into chunks: 

In [35]:
def story_splitter(parsed_output):
    precipitation_output = parsed_output[parsed_output.name.str.contains("annual|wettest|water")]
    dry_output = parsed_output[parsed_output.name.str.contains("dry|snowy|wildfire")]
    storm_output = parsed_output[parsed_output.name.str.contains("storm")]

    return precipitation_output, dry_output, storm_output
    

In [36]:
a, b, c = story_splitter(location_output)

In [40]:
list = [a, b, c]

In [43]:
[letter for letter in list]

[  address   country                                  name midValue     unit
 0  Denver  Colorado  Change in total annual precipitation     -7.0       mm
 1  Denver  Colorado             Change in wettest 90 days     -2.0       mm
 7  Denver  Colorado               Change in water balance     -0.5  z-score,
   address   country                            name midValue  unit
 2  Denver  Colorado          Change in dry hot days     14.0  days
 3  Denver  Colorado            Change in snowy days     -2.0  days
 6  Denver  Colorado  Change in wildfire danger days     15.0  days,
   address   country                                           name midValue  \
 4  Denver  Colorado   Change in frequency of “1-in-100-year” storm      2.0   
 5  Denver  Colorado  Change in precipitation “1-in-100-year” storm     19.0   
 
             unit  
 4  x as frequent  
 5             mm  ]

### getting streaming output for each 

In [45]:
from openai import OpenAI
import os
from dotenv import load_dotenv
 
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

In [49]:
story_system_prompt = """
         "Hello, Climate Change Assistant. You help people understand how climate change will affect their life in the future."
        "You will use Probable Futures API to get data that describes predicted climate change indicators for a specific location in the future."
        "Once you have the data, you will write a story of 2-3 paragraphs of 25-50 words each telling about how climate change is going to impact life in that location based on the Data and Storytelling tips, and Example Story below."
        Please talk about how climate change is going to have a negative impact on daily life as it exists today." 
        "Don't be too negative, but don't be overly optimistic either. Just be direct and clear.  Don't use technical language."

        "-----"
        "DATA"
        "city	country	name	midValue	unit"
        "2	vandalia	usa	Change in dry hot days	15.0	days"
        "3	vandalia	usa	Change in snowy days	-3.0	days"
        "6	vandalia	usa	Change in wildfire danger days	17.0	days"
        "---"
        "STORYTELLING TIPS"
        "- give clear and specific details about that location"
        "- talk about changes that could impact notable activities and characteristics of that location"
        "- don't end with vague optimistic statements"
        "- talk about the trade-offs and hard decisions people will have to make to adapt"
        "- describe scenes in detail as though describing a picture"
        "- provide additional details when strongly implied by the data, for example a rise in wildfire danger in dry climates or sea level rise in coastal ones"
        "---"
        "EXAMPLE STORY"
        "In Vandalia, climate will shift notably due to the increase of 14 dry hot days a year and -2 fewer snowy days a year." 
        "Less snow and more heat will disrupt the outdoor activities that residents enjoy so much and create a sense of loss."
        "Wildfire danger will increase by 15 days a year and threaten the ability to experience nature which is a hallmark of life in Vandalia."
"""

In [54]:
location_output = get_pf_data_new(address="Marrakesh", country="Morocco")


        mutation {
            getDatasetStatistics(input: { 
        country: "Morocco"
        address: "Marrakesh"
                         warmingScenario: "2.0" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


In [55]:
a, b, c = story_splitter(location_output)
list = [a, b, c]

In [81]:
def streaming_completion(story_system_prompt, prompt):
    prompt = story_system_prompt + "  \n" + "-----DATA: " + str(prompt.to_json()) 
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo-16k", #gpt-3.5-turbo #   gpt-4
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    #print('\n \n starting new chunk \n')

    return stream

In [82]:
outputs = []

for letter in list:
    output = ''
    stream = streaming_completion(story_system_prompt, letter)
    print('\n starting new chunk \n')
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += str(chunk.choices[0].delta.content)
    outputs.append(output)


 starting new chunk 

In Marrakesh, Morocco, the future climate change projections reveal a significant shift in weather patterns. The data indicates a decrease in total annual precipitation by -72mm and a reduction of -36mm in the wettest 90 days. These changes in precipitation patterns will have a negative impact on the region's water balance, with a decrease of -1.3 z-score.

These shifts in precipitation levels will undoubtedly impact the daily lives of Marrakesh's residents. The decrease in rainfall will affect agriculture, making it more challenging for farmers to cultivate crops and sustain their livelihoods. Additionally, the reduced availability of water may lead to water scarcity issues, requiring people to conserve and ration water resources carefully.

Furthermore, the altered climate will also have implications for tourism in Marrakesh. The region's renowned natural landscapes and cultural attractions may experience negative consequences. The aesthetic beauty and lush gre

In [83]:
outputs

["In Marrakesh, Morocco, the future climate change projections reveal a significant shift in weather patterns. The data indicates a decrease in total annual precipitation by -72mm and a reduction of -36mm in the wettest 90 days. These changes in precipitation patterns will have a negative impact on the region's water balance, with a decrease of -1.3 z-score.\n\nThese shifts in precipitation levels will undoubtedly impact the daily lives of Marrakesh's residents. The decrease in rainfall will affect agriculture, making it more challenging for farmers to cultivate crops and sustain their livelihoods. Additionally, the reduced availability of water may lead to water scarcity issues, requiring people to conserve and ration water resources carefully.\n\nFurthermore, the altered climate will also have implications for tourism in Marrakesh. The region's renowned natural landscapes and cultural attractions may experience negative consequences. The aesthetic beauty and lush greenery that draw t

### now adding words + images!

In [77]:
# use this for running locally
import urllib
from PIL import Image

In [78]:
def get_image_response(prompt):
    storyboard_prompt = """
    I am making a video about the impact of climate change and I want you to create storyboards.
    I will share a story with you and want you to create an image based on it.
    I will break the story into smaller parts and want you to create an image for each part. 
    Go for photorealistic quality like you are showing a picture.
    Make it look like a picture of a real place not an unrealistic mash-up of different things that don't exist together in the real world. 
    Do not add text and callout boxes on the image.
    The next story chunk is below.
    """
    response = client.images.generate(
      model="dall-e-3",
      prompt=storyboard_prompt + '\n---------' + '\nSTORY CHUNK:' + '\n' + prompt, #"a white siamese cat"
      size="1024x1024",
      quality="standard",
      n=1,
    )

    return response

In [96]:
location_output = get_pf_data_new(address="Denver", country="Colorado")
a, b, c = story_splitter(location_output)
list = [a, b, c]


        mutation {
            getDatasetStatistics(input: { 
        country: "Colorado"
        address: "Denver"
                         warmingScenario: "2.0" 
                }) {
                datasetStatisticsResponses{
                    datasetId
                    midValue
                    name
                    unit
                    warmingScenario
                    latitude
                    longitude
                    info
                }
            }
        }
    


### the final output 🥇

In [97]:
outputs = []
name = 'Denver_'

for letter in list:
    output = ''
    name += 'a' # kludgy hack during testing only
    stream = streaming_completion(story_system_prompt, letter)
    print('\n starting new chunk \n')
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += str(chunk.choices[0].delta.content)

    print('\n generating image, begin')
    image = get_image_response(output)
    print('\n generating image, complete')
    image_url = image.data[0].url
    filename = "story_chunk_" + str(name) + ".png"
    urllib.request.urlretrieve(image_url, filename)
    img = Image.open(filename) 
    img.show()
    
    outputs.append(output)


 starting new chunk 

In Denver, Colorado, climate change predictions indicate a notable decrease in total annual precipitation by 7 millimeters and a decrease in the intensity of the wettest 90 days by 2 millimeters. These changes in precipitation patterns may result in water scarcity and affect the local ecosystems and agriculture. 

Additionally, there will be a decrease in the water balance by 0.5 z-scores, further exacerbating the strain on water resources. The decrease in water availability will require residents to make difficult choices regarding water usage, potentially impacting daily activities such as gardening, outdoor recreation, and water-dependent industries.

As the climate becomes drier and water becomes scarcer, Denver may also face an increase in the threat of wildfires. These changes in climate are likely to lead to more frequent and longer wildfire danger periods, which will pose a risk to both residents and the natural environment.

In conclusion, the anticipate

# Appendix / Old

# testing with streaming API

In [3]:
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_stream_completions.ipynb
# https://platform.openai.com/docs/api-reference/streaming#chat/create-stream

In [11]:
import json

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv
 
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

In [8]:
client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Can you tell me about the writings and teachings of Dr. Andrew Huberman?"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

Andrew Huberman, Ph.D. is an American neuroscientist and tenured professor in the Department of Neurobiology at the Stanford University School of Medicine. He primarily focuses on the fields of brain development, brain plasticity, and neural regeneration and repair. Huberman is well-known for his science communication efforts to the general public, particularly on topics such as neuroscience-based methods for stress management and performance optimization.

Dr. Huberman's research is often centered around understanding how the brain operates under various conditions and states, including during active vision, eye movement, and perception as well as under conditions of stress and anxiety. His research findings often contribute to the development of pharmaceutical and behavioral interventions for brain-related disorders.

In addition to his robust body of scientific research, Dr. Huberman frequently puts out educational content which helps make the field of neuroscience accessible to the

In [20]:
stream

In [21]:
output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        output += str(chunk.choices[0].delta.content)

In [22]:
output

''

In [23]:
output = '''
Andrew Huberman, Ph.D. is an American neuroscientist and tenured professor in the Department of Neurobiology at the Stanford University School of Medicine. He primarily focuses on the fields of brain development, brain plasticity, and neural regeneration and repair. Huberman is well-known for his science communication efforts to the general public, particularly on topics such as neuroscience-based methods for stress management and performance optimization.

Dr. Huberman's research is often centered around understanding how the brain operates under various conditions and states, including during active vision, eye movement, and perception as well as under conditions of stress and anxiety. His research findings often contribute to the development of pharmaceutical and behavioral interventions for brain-related disorders.

In addition to his robust body of scientific research, Dr. Huberman frequently puts out educational content which helps make the field of neuroscience accessible to the general public. For instance, he runs the Huberman Lab Podcast where he shares scientifically proven methods to improve mental and physical health, interviews experts from various fields, and discusses important brain-related topics.

Although Dr. Huberman does not have any published books, he has published numerous academic papers and articles that dive into complex neuroscience topics, such as his work in the areas of neuroplasticity and vision science. He’s also co-founded a company called Boundless, which develops digital therapeutics aiming to improve mental health treatment.

Dr. Huberman is recognized for promoting an understanding of complicated scientific concepts in a manner that can be understood by a broad audience. His teachings often emphasize the idea that people have more control over their brain health, mental health, and overall well-being than they believe. His goal is to help individuals understand the potential significance of their own behaviors, habits, and actions on their brain function and mental health.
'''

In [24]:
output = output.split('\n')

In [25]:
output

['',
 'Andrew Huberman, Ph.D. is an American neuroscientist and tenured professor in the Department of Neurobiology at the Stanford University School of Medicine. He primarily focuses on the fields of brain development, brain plasticity, and neural regeneration and repair. Huberman is well-known for his science communication efforts to the general public, particularly on topics such as neuroscience-based methods for stress management and performance optimization.',
 '',
 "Dr. Huberman's research is often centered around understanding how the brain operates under various conditions and states, including during active vision, eye movement, and perception as well as under conditions of stress and anxiety. His research findings often contribute to the development of pharmaceutical and behavioral interventions for brain-related disorders.",
 '',
 'In addition to his robust body of scientific research, Dr. Huberman frequently puts out educational content which helps make the field of neurosc

In [71]:
output[(output['name'].str.contains('Change')) & (output['city'].str.contains('denver'))]

,datasetId,midValue,name,unit,warmingScenario,latitude,longitude,climateZoneName,city,country
0,40601,-7.0,Change in total annual precipitation,mm,2.0,39.8,-105,NaN,denver,denver
1,40616,-2.0,Change in wettest 90 days,mm,2.0,39.8,-105,NaN,denver,denver
2,40607,14.0,Change in dry hot days,days,2.0,39.8,-105,NaN,denver,denver
3,40614,-2.0,Change in snowy days,days,2.0,39.8,-105,NaN,denver,denver
4,40612,2.0,Change in frequency of “1-in-100-year” storm,x as frequent,2.0,39.8,-105,NaN,denver,denver
7,40613,19.0,Change in precipitation “1-in-100-year” storm,mm,2.0,39.8,-105,NaN,denver,denver
10,40704,15.0,Change in wildfire danger days,days,2.0,39.8,-105,NaN,denver,denver
11,40703,-0.5,Change in water balance,z-score,2.0,39.8,-105,NaN,denver,denver


In [72]:
output[(output['name'].str.contains('Change')) & (output['city'].str.contains('miami'))]

,datasetId,midValue,name,unit,warmingScenario,latitude,longitude,climateZoneName,city,country
54,40601,99.0,Change in total annual precipitation,mm,2.0,25.8,-80.2,NaN,miami,miami
55,40616,61.0,Change in wettest 90 days,mm,2.0,25.8,-80.2,NaN,miami,miami
56,40607,18.0,Change in dry hot days,days,2.0,25.8,-80.2,NaN,miami,miami
57,40614,0.0,Change in snowy days,days,2.0,25.8,-80.2,NaN,miami,miami
58,40612,2.0,Change in frequency of “1-in-100-year” storm,x as frequent,2.0,25.8,-80.2,NaN,miami,miami
61,40613,86.0,Change in precipitation “1-in-100-year” storm,mm,2.0,25.8,-80.2,NaN,miami,miami
64,40704,9.0,Change in wildfire danger days,days,2.0,25.8,-80.2,NaN,miami,miami
65,40703,0.0,Change in water balance,z-score,2.0,25.8,-80.2,NaN,miami,miami


In [73]:
output[(output['name'].str.contains('Change')) & (output['city'].str.contains('kyoto'))]

,datasetId,midValue,name,unit,warmingScenario,latitude,longitude,climateZoneName,city,country
27,40601,65.0,Change in total annual precipitation,mm,2.0,35.0,135.8,NaN,kyoto,kyoto
28,40616,52.0,Change in wettest 90 days,mm,2.0,35.0,135.8,NaN,kyoto,kyoto
29,40607,6.0,Change in dry hot days,days,2.0,35.0,135.8,NaN,kyoto,kyoto
30,40614,-2.0,Change in snowy days,days,2.0,35.0,135.8,NaN,kyoto,kyoto
31,40612,2.0,Change in frequency of “1-in-100-year” storm,x as frequent,2.0,35.0,135.8,NaN,kyoto,kyoto
34,40613,51.0,Change in precipitation “1-in-100-year” storm,mm,2.0,35.0,135.8,NaN,kyoto,kyoto
37,40704,6.0,Change in wildfire danger days,days,2.0,35.0,135.8,NaN,kyoto,kyoto
38,40703,0.2,Change in water balance,z-score,2.0,35.0,135.8,NaN,kyoto,kyoto
